# NSW  Sensitive List

## List Identifiers
* Sensitive list: __[dr487](https://lists.ala.org.au/speciesListItem/list/dr487)__ (and [dr18457 in test](https://lists-test.ala.org.au/speciesListItem/list/dr18457))

* Collection:  __[dr487](https://collections.ala.org.au/public/show/dr487)__
### Source Data

*  __[Home Page](https://data.bionet.nsw.gov.au)__
*  __[API/Data Info](https://data.bionet.nsw.gov.au/biosvcapp/odata/SpeciesNames)__
*  __[Data Information and file downloads](https://www.environment.nsw.gov.au/research-and-publications/publications-search/bionet-sensitive-species-list)__


**Metadata Description**
The NSW Government [http://www.environment.nsw.gov.au Office of Environment & Heritage] (OEH) maintains the Atlas of NSW Wildlife, a database of recorded sightings of flora and fauna species in the state. The Atlas contains records of plants, mammals, birds, reptiles, amphibians and some fish, fauna and invertebrates. The OEH has created a sensitive species data policy that sets out the manner in which 'sensitive' species will be categorised, and the levels of disclosure that can be allowed. See [http://www.environment.nsw.gov.au/policiesandguidelines/SensitiveSpeciesPolicy.htm more details of the policy]. The Atlas of Living Australia generalises latitude and longitude values in occurrence records for species in Category 2 to 1 decimal place (10km) and those in Category 3 to 2 decimal places (1km). Latitude and longitude values are withheld for species in Category 1. These rules are applied to all NSW occurrence records, regardless of source. See [http://biocache.ala.org.au/occurrence/0583b88e-6aa3-432e-9154-9b1cbaa59f74 an example record]. To access sensitive data for species in this list, email the BioNet mailbox at bionet@environment.nsw.gov.au and address it to the Data Exchange Officer requesting permission.

Part of the sensitive data service - Yes
Region - NSW
SDS Authority - NSW OEH
SDS Type - CONSERVATION

**Metadata URL**
https://data.bionet.nsw.gov.au

## Setup and Download source
Import required libraries
Retrieve dataset from API
Set Project directory

In [1]:
import pandas as pd
import urllib.request, json
import os
import sys
import datetime
import ssl

#projectDir = "/Users/oco115/PycharmProjects/authoritative-lists/"
projectDir = "/Users/new330/IdeaProjects/authoritative-lists/"
statusDir = projectDir + "source-data/status-codes/"
dateStr = datetime.datetime.now().strftime('%Y%m%d')
sys.path.append(os.path.abspath(projectDir + "source-code/includes"))
import list_functions as lf

In [2]:
# %%script echo skipping # comment this line to download the files and save locally

with urllib.request.urlopen("https://data.bionet.nsw.gov.au/biosvcapp/odata/SpeciesNames",context=ssl._create_unverified_context()) as url:
    data = json.loads(url.read().decode())
nswspeciesnames = pd.json_normalize(data, record_path =['value'])
nswspeciesnames.to_csv(projectDir + "source-data/NSW/SpeciesNames.csv",encoding="UTF-8",index=False)
nswspeciesnames.head(5)

,dcterms_rightsHolder,dcterms_rights,dcterms_language,dcterms_type,dcterms_modified,dcterms_available,speciesID,taxonRank,kingdomID,kingdom,...,stateConservation,protectedInNSW,sensitivityClass,TSProfileID,countryConservation,highThreatWeed,widelyCultivatedNativeSpecies,CAMBA,JAMBA,ROKAMBA
0,"NSW Dept of Planning, Industry and Environment",CC-BY 4.0,en,service,2013-12-20T15:06:07.13+11:00,1995-12-15T12:48:13+11:00,1,Species,138,Animalia,...,Vulnerable,true,Not Sensitive,10211.0,Vulnerable,None,None,false,false,false
1,"NSW Dept of Planning, Industry and Environment",CC-BY 4.0,en,service,2006-01-30T17:20:32+11:00,1995-12-15T12:48:13+11:00,2,Species,138,Animalia,...,Not Listed,true,Not Sensitive,NaN,Not Listed,None,None,false,false,false
2,"NSW Dept of Planning, Industry and Environment",CC-BY 4.0,en,service,2006-11-23T16:57:26+11:00,1995-12-15T12:48:14+11:00,3,Species,138,Animalia,...,Not Listed,true,Not Sensitive,NaN,Not Listed,None,None,false,false,false
3,"NSW Dept of Planning, Industry and Environment",CC-BY 4.0,en,service,2006-01-30T17:20:32+11:00,1995-12-15T12:48:14+11:00,4,Species,138,Animalia,...,Not Listed,true,Not Sensitive,NaN,Not Listed,None,None,false,false,false
4,"NSW Dept of Planning, Industry and Environment",CC-BY 4.0,en,service,2021-02-01T18:14:41.78+11:00,1995-12-15T12:48:14+11:00,5,Species,138,Animalia,...,Not Listed,true,Not Sensitive,NaN,Not Listed,None,None,false,false,false


In [32]:
# 1
dictn = {"Category 3": "1km",
       "Category 2": "10km",
       "Category 1": "WITHHOLD"
}
select = ['Category 3','Category 2','Category 1']
sensitivelist = nswspeciesnames
sensitivelist['generalisation'] = sensitivelist['sensitivityClass']
sensitivelist['generalisation'] = sensitivelist['generalisation'].replace(dictn)

# 2
sensitivelist = sensitivelist[(sensitivelist['speciesID'] == sensitivelist['taxonID']) & sensitivelist['sensitivityClass'].isin(select)]
sensitivelist = sensitivelist[['scientificName', 'family', 'genus', 'vernacularName','generalisation', 'sensitivityClass']]
sensitivelist = sensitivelist.rename(columns={'sensitivityClass': 'category'})

# 3
sensitivelist.to_csv(projectDir + "current-lists/sensitive-lists/NSW-sensitive-" + dateStr + ".csv",encoding="UTF-8",index=False)
sensitivelist.head(5)

,scientificName,family,genus,vernacularName,generalisation,category
5,Callocephalon fimbriatum,Cacatuidae,Callocephalon,Gang-gang Cockatoo,1km,Category 3
51,Anthochaera phrygia,Meliphagidae,Anthochaera,Regent Honeyeater,10km,Category 2
159,Ninox strenua,Strigidae,Ninox,Powerful Owl,1km,Category 3
326,Tyto tenebricosa,Tytonidae,Tyto,Sooty Owl,1km,Category 3
356,Ninox connivens,Strigidae,Ninox,Barking Owl,1km,Category 3


In [33]:
## 22/08/2023
# add an item to the list
# NSW have added a subspecies level entry (Calyptorhynchus lathami lathami) and replaced it with a species entry (Calyptorhynchus lathami), which the SDS will not work with, because it doesn't necessarily cascade changes. Both need to be in the list.
# Add the species entry back to the list, upload it to test, and run the changelist process again
new = sensitivelist[sensitivelist.scientificName == "Calyptorhynchus lathami lathami"].copy()
new[['scientificName']] = "Calyptorhynchus lathami" # replace the name
new[['vernacularName']] = "Glossy Black-cockatoo"
sensitivelist = pd.concat([sensitivelist,new])
sensitivelist[sensitivelist['scientificName'].str.startswith('Calyptorhynchus lathami')]
sensitivelist.to_csv(projectDir + "current-lists/sensitive-lists/NSW-sensitive-" + dateStr + ".csv",encoding="UTF-8",index=False)

## Write change logs
Upload the file to the test environment before running the below, to check the difference between the list in test with that in production.
- check record counts old vs new
- check change log adds up
- send to domain experts for verification

In [34]:
ltype = "S"
changeDir = "Monitoring/Change-logs/"
filename = "NSW-sensitive.csv"
testdr = "dr18457"
proddr = "dr487"
changelist = lf.get_changelist(testdr, proddr, ltype)
changelist.to_csv(projectDir + changeDir + dateStr + "-" + filename, encoding="UTF-8", index=False)
changelist

get_changelist: Test -  dr18457 Prod -  dr487
download_ala_list:  https://lists.ala.org.au/ws/speciesListItems/dr487?max=10000&includeKVP=true
Index(['id', 'name', 'commonName', 'scientificName', 'lsid', 'dataResourceUid',
       'kvpValues'],
      dtype='object')
download_ala_list:  https://lists-test.ala.org.au/ws/speciesListItems/dr18457?max=10000&includeKVP=true
Index(['id', 'name', 'commonName', 'scientificName', 'lsid', 'dataResourceUid',
       'kvpValues'],
      dtype='object')


,name,scientificName_old,scientificName_new,commonName_old,commonName_new,listUpdate
148,Calyptorhynchus lathami lathami,NaN,Calyptorhynchus (Calyptorhynchus) lathami lathami,NaN,Glossy Black-cockatoo,added
186,Leionema lamprophyllum subsp. fractum,NaN,Leionema lamprophyllum subsp. fractum,NaN,None,added
199,Danhatchia copelandii,NaN,Danhatchia copelandii,NaN,None,added
139,Leionema lamprophyllum subsp. obovatum,Leionema lamprophyllum subsp. obovatum,NaN,Shiny Leionema,NaN,removed
140,Caladenia porphyrea,Caladenia quadrifaria,NaN,None,NaN,removed


If all ok, run the following to keep a copy of the list in prod before overwriting it

In [36]:
# Sensitive Lists
state= "NSW"
filename = state + "-" + proddr + "-sensitive.csv"
prodListUrl = "https://lists.ala.org.au/ws/speciesListItems/" + proddr + "?max=10000&includeKVP=true"
prodList = lf.download_ala_specieslist(prodListUrl)  # save the prod list to the historical lists directory
prodList = lf.kvp_to_columns(prodList)
prodList.to_csv(projectDir + "historical-lists/sensitive/" + filename, encoding="UTF-8", index=False)
print('Finished downloading sensitive historical list')

download_ala_list:  https://lists.ala.org.au/ws/speciesListItems/dr487?max=10000&includeKVP=true
Index(['id', 'name', 'commonName', 'scientificName', 'lsid', 'dataResourceUid',
       'kvpValues'],
      dtype='object')
Finished downloading sensitive historical list
